Redis

In [ ]:
Redis 是速度非常快的非关系型（NoSQL）内存键值数据库

In [ ]:
键的类型只能为字符串，值支持五种数据类型：字符串、列表、集合、散列表、有序集合。

In [ ]:
拉链法
为避免不同关键字会被映射到同一个地址空间中，便将所有同义词存储在同一个线性链表中

In [ ]:
使用场景

计数器
可以对 String 进行自增自减运算，从而实现计数器功能。
Redis 这种内存型数据库的读写性能非常高，很适合存储频繁读写的计数量。

缓存
将热点数据放到内存中，设置内存的最大使用量以及淘汰策略来保证缓存的命中率（从缓存中成功获取数据的比例）。

查找表
查找表和缓存类似，也是利用了 Redis 快速的查找特性。但是查找表的内容不能失效，而缓存的内容可以失效，因为缓存不作为可靠的数据来源。

消息队列
List 是一个双向链表，可以通过 lpush 和 rpop 写入和读取消息

In [ ]:
Redis 可以为每个键设置过期时间，当键过期时，会自动删除该键。

In [ ]:
持久化
Redis 是内存型数据库，为了保证数据在断电后不会丢失，需要将内存中的数据持久化到硬盘上。

In [ ]:
一个事务包含了多个命令，服务器在执行事务期间，不会改去执行其它客户端的命令请求。
事务中的多个命令被一次性发送给服务器，而不是一条一条发送，这种方式被称为流水线，它可以减少客户端与服务器之间的网络通信次数从而提升性能。
Redis 最简单的事务实现方式是使用 MULTI 和 EXEC 命令将事务操作包围起来。

In [ ]:
一个从服务器只能有一个主服务器，并且不支持主主复制。
Sentinel（哨兵）可以监听集群中的服务器，并在主服务器进入下线状态时，自动从从服务器中选举出新的主服务器。